In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import scipy
import warnings
import time

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import CCA, PLSCanonical
from sklearn.utils import Bunch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

from cca_zoo.models import CCA as cz_CCA
from cca_zoo.models import rCCA as cz_rCCA
from cca_zoo.model_selection import GridSearchCV as cz_GridSearchCV
from cca_zoo.plotting import pairplot_train_test

rs_num = 42
rng = np.random.default_rng(rs_num)

sklearn.set_config(display="diagram")

n_splits = 5
n_repeats = 50

%matplotlib inline

# 1. Loading Data

In [2]:
# load data
data_path = 'data//'
X_train = np.genfromtxt(data_path+'example_2_X_a.csv', delimiter=',')
Y_train = np.genfromtxt(data_path+'example_2_X_b.csv', delimiter=',')
X_test = np.genfromtxt(data_path+'example_2_X_a_test.csv', delimiter=',')
Y_test = np.genfromtxt(data_path+'example_2_X_b_test.csv', delimiter=',')

# extract dimensions
X_dimension_num = X_train.shape[1]
Y_dimension_num = Y_train.shape[1]

In [3]:
# check if data are standardized
print('For X_train, ')
print('  mean:', X_train.mean(axis=0))
print('  std:', X_train.std(axis=0, ddof=1))

print('For Y_train, ')
print('  mean:', Y_train.mean(axis=0))
print('  std:', Y_train.std(axis=0, ddof=1))

For X_train, 
  mean: [ 1.45716772e-16 -1.29757316e-16  2.08166817e-17  1.97064587e-16]
  std: [1. 1. 1. 1.]
For Y_train, 
  mean: [ 3.27515792e-16 -6.66133815e-17 -4.80171458e-16]
  std: [1. 1. 1.]


# 2. Example 2.1 - Solving Standard Eigenvalue Problem

## 2.1. Covariance Matrix

In [4]:
C = np.cov(X_train.T, Y_train.T, ddof=1)
print('For C:')
print(np.array_str(C, precision=2))

For C:
[[ 1.    0.34 -0.11  0.21 -0.1   0.92 -0.21]
 [ 0.34  1.   -0.08  0.03 -0.1   0.34  0.06]
 [-0.11 -0.08  1.   -0.3   0.98 -0.03  0.3 ]
 [ 0.21  0.03 -0.3   1.   -0.25  0.12 -0.94]
 [-0.1  -0.1   0.98 -0.25  1.   -0.03  0.25]
 [ 0.92  0.34 -0.03  0.12 -0.03  1.   -0.13]
 [-0.21  0.06  0.3  -0.94  0.25 -0.13  1.  ]]


In [5]:
Cxx = C[0:X_dimension_num, 0:X_dimension_num]
Cxy = C[0:X_dimension_num, X_dimension_num:]
Cyy = C[X_dimension_num:, X_dimension_num:]

C_compute = np.linalg.multi_dot(
    [np.linalg.inv(Cyy), Cxy.T, np.linalg.inv(Cxx), Cxy]
)

In [6]:
print('For Cxx:')
print(np.array_str(Cxx, precision=2))

For Cxx:
[[ 1.    0.34 -0.11  0.21]
 [ 0.34  1.   -0.08  0.03]
 [-0.11 -0.08  1.   -0.3 ]
 [ 0.21  0.03 -0.3   1.  ]]


In [7]:
print('For Cxy:')
print(np.array_str(Cxy, precision=2))

For Cxy:
[[-0.1   0.92 -0.21]
 [-0.1   0.34  0.06]
 [ 0.98 -0.03  0.3 ]
 [-0.25  0.12 -0.94]]


In [8]:
print('For Cyy:')
print(np.array_str(Cyy, precision=2))

For Cyy:
[[ 1.   -0.03  0.25]
 [-0.03  1.   -0.13]
 [ 0.25 -0.13  1.  ]]


## 2.2. Compute Weights and Canonical Correlation

In [9]:
# standard eigenvalue problem
std_eig_bunch = Bunch()
std_eig_bunch.name = 'std eig'

# solve eig
[std_eig_bunch.eig_val, 
 std_eig_bunch.eig_vec] = np.linalg.eig(C_compute)

# Y matrix weights
std_eig_bunch.wy = std_eig_bunch.eig_vec

# canonical correlation array
std_eig_bunch.cc_arr = np.sqrt(std_eig_bunch.eig_val)

# X matrix weights
std_eig_bunch.wx = np.linalg.multi_dot(
    [np.linalg.inv(Cxx), Cxy, std_eig_bunch.wy]
)/std_eig_bunch.cc_arr

# sort by canonical correlation
sort_ind = std_eig_bunch.cc_arr.argsort() # ascending order; use [::-1] for descending order
std_eig_bunch.cc_arr_sorted = std_eig_bunch.cc_arr[sort_ind[::-1]]
std_eig_bunch.wx_sorted = std_eig_bunch.wx[:, sort_ind[::-1]]
std_eig_bunch.wy_sorted = std_eig_bunch.wy[:, sort_ind[::-1]]

del sort_ind

In [10]:
print('Weight vector w_x calculated by Standard Eigenvalue Problem:')
print(np.array_str(std_eig_bunch.wx_sorted, precision=2, suppress_small=True))

Weight vector w_x calculated by Standard Eigenvalue Problem:
[[-0.04 -0.41 -0.84]
 [-0.    0.09 -0.1 ]
 [-0.99 -0.41  0.14]
 [ 0.18 -0.83  0.52]]


In [11]:
print('Weight vector w_y calculated by Standard Eigenvalue Problem:')
print(np.array_str(std_eig_bunch.wy_sorted, precision=2, suppress_small=True))

Weight vector w_y calculated by Standard Eigenvalue Problem:
[[-0.97 -0.39  0.19]
 [-0.04 -0.37 -0.86]
 [-0.22  0.85 -0.46]]


In [12]:
print('Canonical Correlation calculated by Standard Eigenvalue Problem:')
print(np.array_str(std_eig_bunch.cc_arr_sorted, precision=3, suppress_small=True))

Canonical Correlation calculated by Standard Eigenvalue Problem:
[0.985 0.939 0.922]


# 3. Example 2.2. - Solving Generalized Eigenvalue Problem

In [13]:
gen_eig_bunch = Bunch()
gen_eig_bunch.name = 'gen eig'

# construct general matrices
gen_eig_bunch.A = np.zeros(
    (
        X_dimension_num+Y_dimension_num, 
        X_dimension_num+Y_dimension_num
    )
)
gen_eig_bunch.A[:X_dimension_num, X_dimension_num:] = Cxy
gen_eig_bunch.A[X_dimension_num:, :X_dimension_num] = Cxy.T

gen_eig_bunch.B = np.zeros_like(gen_eig_bunch.A)
gen_eig_bunch.B[:X_dimension_num, :X_dimension_num] = Cxx
gen_eig_bunch.B[X_dimension_num:, X_dimension_num:] = Cyy

# solve generalized eigenvalue problem
[gen_eig_bunch.eig_val, 
 gen_eig_bunch.eig_vec] = scipy.linalg.eigh(gen_eig_bunch.A, gen_eig_bunch.B)

# retrive canonical correlation and weight vectors
gen_eig_bunch.cc_arr = gen_eig_bunch.eig_val[gen_eig_bunch.eig_val>=1e-6]
gen_eig_bunch.wx = gen_eig_bunch.eig_vec[:X_dimension_num, gen_eig_bunch.eig_val>=1e-6]
gen_eig_bunch.wy = gen_eig_bunch.eig_vec[X_dimension_num:, gen_eig_bunch.eig_val>=1e-6]

# sort out w.r.t. cc values
sort_ind = gen_eig_bunch.cc_arr.argsort()
gen_eig_bunch.cc_arr_sorted = gen_eig_bunch.cc_arr[sort_ind[::-1]]
gen_eig_bunch.wx_sorted = gen_eig_bunch.wx[:, sort_ind[::-1]]
gen_eig_bunch.wy_sorted = gen_eig_bunch.wy[:, sort_ind[::-1]]

del sort_ind

In [14]:
print('The generalized eigenvalues:')
print(np.array_str(gen_eig_bunch.eig_val, precision=4, suppress_small=True))

The generalized eigenvalues:
[-0.985  -0.9388 -0.9222  0.      0.9222  0.9388  0.985 ]


In [15]:
print('Weight vector w_x calculated by Generalized Eigenvalue Problem:')
print(np.array_str(gen_eig_bunch.wx_sorted, precision=3, suppress_small=True))

Weight vector w_x calculated by Generalized Eigenvalue Problem:
[[-0.026  0.304 -0.64 ]
 [-0.    -0.069 -0.074]
 [-0.666  0.302  0.103]
 [ 0.119  0.616  0.395]]


If you look above, the eigenvectors are not quite similar to the ones in the paper, probably because the eigenvectors are not unique.
If we normalize them, they will be much similar to what we have in the paper.

In [16]:
print(
    np.array_str(
        normalize(gen_eig_bunch.wx_sorted, axis=0), 
        precision=3, suppress_small=True
    )
)

[[-0.038  0.403 -0.839]
 [-0.001 -0.092 -0.096]
 [-0.984  0.401  0.135]
 [ 0.176  0.817  0.518]]


In [17]:
print('Weight vector w_y calculated by Generalized Eigenvalue Problem:')
print(np.array_str(gen_eig_bunch.wy_sorted, precision=3, suppress_small=True))

Weight vector w_y calculated by Generalized Eigenvalue Problem:
[[-0.655  0.286  0.147]
 [-0.025  0.271 -0.659]
 [-0.151 -0.628 -0.353]]


In [18]:
print('Canonical Correlation calculated by Generalized Eigenvalue Problem:')
print(np.array_str(gen_eig_bunch.cc_arr_sorted, precision=3, suppress_small=True))

Canonical Correlation calculated by Generalized Eigenvalue Problem:
[0.985 0.939 0.922]


# 4. Example 2.3 - Solving SVD

In [19]:
svd_bunch = Bunch()
svd_bunch.name = 'svd'

svd_bunch.Cxx_sqrt_inv = scipy.linalg.inv(
    scipy.linalg.sqrtm(Cxx)
) # lower Cholesky decomposition
svd_bunch.Cyy_sqrt_inv = scipy.linalg.inv(
    scipy.linalg.sqrtm(Cyy)
)

svd_bunch.mat_to_svd = np.linalg.multi_dot(
    [svd_bunch.Cxx_sqrt_inv, Cxy, svd_bunch.Cyy_sqrt_inv]
) # matrix to SVD

# SVD decomposition
[svd_bunch.U, 
 svd_bunch.cc_arr_sorted, 
 svd_bunch.Vh] = scipy.linalg.svd(
    svd_bunch.mat_to_svd, full_matrices=False
)

# compute weights vectors
svd_bunch.wx_sorted = np.matmul(svd_bunch.Cxx_sqrt_inv, svd_bunch.U)
svd_bunch.wy_sorted = np.matmul(svd_bunch.Cyy_sqrt_inv, svd_bunch.Vh.T)

In [20]:
print('The matrix to be SVD:')
print(np.array_str(svd_bunch.mat_to_svd, precision=2, suppress_small=True))

The matrix to be SVD:
[[-0.02  0.9  -0.06]
 [-0.07  0.2   0.12]
 [ 0.98  0.04  0.04]
 [ 0.01 -0.02 -0.93]]


In [21]:
print('The U.T')
print(np.array_str(svd_bunch.U.T, precision=2, suppress_small=True))

The U.T
[[-0.03 -0.03  0.95 -0.3 ]
 [-0.47  0.03 -0.28 -0.84]
 [-0.85 -0.26  0.11  0.44]]


In [22]:
print('The V')
print(np.array_str(svd_bunch.Vh.T, precision=2, suppress_small=True))

The V
[[ 0.95 -0.29  0.15]
 [ 0.01 -0.44 -0.9 ]
 [ 0.33  0.85 -0.41]]


In [23]:
print('Weight vector w_x calculated by SVD:')
print(np.array_str(svd_bunch.wx_sorted, precision=3, suppress_small=True))

Weight vector w_x calculated by SVD:
[[ 0.036 -0.43  -0.906]
 [ 0.001  0.098 -0.104]
 [ 0.942 -0.427  0.145]
 [-0.168 -0.871  0.559]]


In [24]:
print('Weight vector w_y calculated by SVD:')
print(np.array_str(svd_bunch.wy_sorted, precision=3, suppress_small=True))

Weight vector w_y calculated by SVD:
[[ 0.926 -0.405  0.208]
 [ 0.036 -0.384 -0.932]
 [ 0.214  0.888 -0.499]]


In [25]:
print('Canonical Correlation calculated by SVD:')
print(np.array_str(svd_bunch.cc_arr_sorted, precision=3, suppress_small=True))

Canonical Correlation calculated by SVD:
[0.985 0.939 0.922]
